In [1]:

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from bson.json_util import dumps

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import 
from crud import AnimalShelter  

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here.
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Div(
        dcc.RadioItems(
            id='rd',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue ', 'value': 'Disaster Rescue or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Water Rescue',
            labelStyle={'display': 'inline-block'}
        )
    ),
    html.Hr(),
    html.Div(id='dd-output-container', children=[
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            editable=True,
            style_cell={'textAlign': 'left'},
            style_cell_conditional=[
                {
                    'if': {'column_id': s},
                    'textAlign': 'left'
                } for s in ['animal_type', 'color', 'breed', 'animal_id']
            ],
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(0, 128, 0)'
                }
            ],
            style_header={
                'backgroundColor': 'rgb(0, 238, 0)',
                'fontWeight': 'bold'
            },
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable="single",
            row_selectable="multi",
            row_deletable=True,
            selected_columns=[],
            selected_rows=[],
            page_action="native",
            page_current=0,
            page_size=10,
        )
    ]),
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('dd-output-container', 'children'),
    [Input('rd', 'value')]
)
def update_output(value):
    # Data filtering logic goes here
    # This is a placeholder implementation. Replace with your own logic.
    if value == 'Water Rescue':
        # Example filter query
        df_filtered = pd.DataFrame.from_records(db.read({}))
    else:
        df_filtered = pd.DataFrame.from_records(db.read({}))

    return [
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df_filtered.columns],
            data=df_filtered.to_dict('records'),
            # Rest of the DataTable properties
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
    # Map updating logic goes here
    # This is a placeholder implementation. Replace with your own logic.
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[30.75, -97.48])
        ])
    ]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    # Graph updating logic goes here
    # This is a placeholder implementation. Replace with your own logic.
    return [
        dcc.Graph(
            figure=px.bar()  # Placeholder figure
        )
    ]

# Additional callback functions can be added as needed

app.run_server(debug=True)


ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 65a77517d15e555d170aaa60, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>